In [ ]:
import os
import numpy as np
import pandas as pd
import json
os.environ["NAMESPACE"]="research"
os.environ["PROFILE"]="local"
from agrobrain_util.runtime.evironment import RuntimeEnv
from agrobrain_util.infra.app_config import application_config as cfg

import shutil
import glob
from skimage import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from shapely.geometry import Point, Polygon

import json
import random
import glob
import datetime
from tqdm import tqdm
from datetime import datetime

from skimage.transform import resize
from skimage.filters import gaussian
import skimage.measure

from PIL import Image

import dtlpy as dl
if dl.token_expired():
    dl.login()

env = RuntimeEnv()

DATA_DIR = "data"



project = dl.projects.get(project_name='Taranis AI Annotation Projects')


In [ ]:
# DOWNLOAD ANNOTATIONS


def download_datasets_annotations_from_dataloop(dataset_name, task_name, version):
    dataloop_local_data_dir = os.path.join(DATA_DIR, f"dataloop")
    annotation_local_path = os.path.join(dataloop_local_data_dir, f"annotations_{dataset_name}_task_{task_name}_v{version}")
    dataset = project.datasets.get(dataset_name=dataset_name)
    dataset.download_annotations(local_path=annotation_local_path)
    print(f"Done downloading annotations for the dataset: {dataset_name}.\n Local path: {annotation_local_path}")


def get_jsons_paths_list(dataset_name, task_name, version, sub_folder=""):
    dataloop_local_data_dir = os.path.join(DATA_DIR, f"dataloop")
    annotation_local_path = os.path.join(dataloop_local_data_dir, f"annotations_{dataset_name}_task_{task_name}_v{version}")
    if sub_folder=="":
        jsons_folder = os.path.join(annotation_local_path, "json")
    else:
        jsons_folder = os.path.join(annotation_local_path, "json", sub_folder)
    jsons_paths_list = glob.glob(os.path.join(jsons_folder, "*.json"))
    return jsons_paths_list


# POINT TAGS

POINTS_DATASET_NAME = "anafa_2023_07_12_resolution_lim_5_res_20_images"
POINTS_TASK_NAME = 'anafa_2023_07_12_resolution_lim_5_res_20_images'
POINTS_VERSION = 0

# download_datasets_annotations_from_dataloop(POINTS_DATASET_NAME, POINTS_TASK_NAME, POINTS_VERSION)
points_jsons_paths_list = get_jsons_paths_list(POINTS_DATASET_NAME, POINTS_TASK_NAME, POINTS_VERSION, sub_folder="resized_20_images")
points_task = project.tasks.get(task_name=POINTS_TASK_NAME)


# POLYGONS TAGS

POLYGONS_DATASET_NAME = "anafa_2023_06_23_resolution_lim_dataset_1"
POLYGONS_TASK_NAME = 'anafa_2023_06_23_resolution_lim_first_task_01'
POLYGONS_VERSION = 0

# download_datasets_annotations_from_dataloop(POLYGONS_DATASET_NAME, POLYGONS_TASK_NAME, POLYGONS_VERSION)
polygons_jsons_paths_list = get_jsons_paths_list(POLYGONS_DATASET_NAME, POLYGONS_TASK_NAME, POLYGONS_VERSION)
polygons_task = project.tasks.get(task_name=POLYGONS_TASK_NAME)


points_task_image_ids_list = [int(os.path.basename(p).split("_")[0]) for p in points_jsons_paths_list]
filtered_polygons_jsons_paths_list = [elem for elem in polygons_jsons_paths_list if any(str(id_) in os.path.basename(elem) for id_ in points_task_image_ids_list)]

len(points_jsons_paths_list), len(polygons_jsons_paths_list), len(filtered_polygons_jsons_paths_list)

In [ ]:
# READ TAGS OF ALL THE IMAGES

for image_id in points_task_image_ids_list:
    points_tags_json = [path for path in points_jsons_paths_list if str(image_id) in path][0]
    polygons_tags_json = [path for path in polygons_jsons_paths_list if str(image_id) in path][0]

    with open(points_tags_json) as file:
        points_json_data = json.load(file)

    with open(polygons_tags_json) as file:
        polygons_json_data = json.load(file)

    


In [ ]:
# EXAMPLE - DISPLAY ONE IMAGE TAGS

example_image_id = 9638024
example_im_path = env.download_image(int(example_image_id))
example_image = io.imread(example_im_path)
original_image_width = example_image.shape[1]
original_image_height = example_image.shape[0]

# image_id = points_task_image_ids_list[15] 
image_id = 9638024
# image_id = 6580458
# image_id = 9445268
points_tags_json = [path for path in points_jsons_paths_list if str(image_id) in path][0]
polygons_tags_json = [path for path in polygons_jsons_paths_list if str(image_id) in path][0]


im_path = env.download_image(int(image_id))
image = io.imread(im_path)


with open(points_tags_json) as file:
    points_json_data = json.load(file)

with open(polygons_tags_json) as file:
    polygons_json_data = json.load(file)



fig, ax = plt.subplots(figsize=(20,10))

ax.imshow(image)

shapely_polygons = []

# DRAW POLYGONS
for i in range(len(polygons_json_data['annotations'])):
    x_values = [coord['x'] for coord in polygons_json_data['annotations'][i]['coordinates'][0]]
    y_values = [coord['y'] for coord in polygons_json_data['annotations'][i]['coordinates'][0]]
    polygon = patches.Polygon(list(zip(x_values, y_values)), closed=True, fill=None, edgecolor='aquamarine')
    shapely_polygons.append(Polygon(list(zip(x_values, y_values))))
    ax.add_patch(polygon)
    ax.set_title(f"image: {im_path}")


# DRAW POINTS
for i in range(len(points_json_data['annotations'])):
    width = points_json_data['metadata']['system']['width']
    height = points_json_data['metadata']['system']['height']

    width_factor = original_image_width / width
    height_factor = original_image_height / height

    x_value = int(points_json_data['annotations'][i]['coordinates']['x']) * width_factor
    y_value = int(points_json_data['annotations'][i]['coordinates']['y']) * height_factor

    shapely_point = Point(x_value, y_value)

    is_inside = np.any([poly.contains(shapely_point) for poly in shapely_polygons])
    if is_inside:
        point_color = 'mo'
    else:
        point_color = 'ro'

    ax.plot(x_value, y_value, point_color)

    ax.set_xlim(3900, 4500)
    ax.set_ylim(1300, 500)
    # ax.set_xlim(1800, 2100)
    # ax.set_ylim(2800, 2500)


    ax.set_title(f"image: {im_path}. {len(points_json_data['annotations'])} point tags")


plt.show()

In [ ]:
# CALCULATE ALL POLYGONS AREA

example_image_id = int(os.path.basename(jsons_paths_list[0]).replace(".json", ""))
im_path = env.download_image(int(example_image_id))
image = io.imread(im_path)
image_shape = image.shape
image_area = image_shape[0] * image_shape[1]

polygons_areas_pixels = {}
polygons_areas = {}


for i in range(len(jsons_paths_list)):
    json_path = jsons_paths_list[i]
    image_id = int(os.path.basename(json_path).replace(".json", ""))
    image_data = images_data[images_data['imageID'] == image_id]

    with open(json_path) as file:
        data = json.load(file)

    image_polygons_areas_pixels = []
    image_polygons_areas = []
    for i in range(len(data['annotations'])):
        x_values = [coord['x'] for coord in data['annotations'][i]['coordinates'][0]]
        y_values = [coord['y'] for coord in data['annotations'][i]['coordinates'][0]]
        polygon = patches.Polygon(list(zip(x_values, y_values)), closed=True, fill=None, edgecolor='aquamarine')
        pgon = Polygon(zip(x_values, y_values))
        image_polygons_areas_pixels.append(pgon.area)
        image_polygons_areas.append((pgon.area/image_area)* image_data['footprintArea'].values[0])
        
    polygons_areas_pixels[image_id] = image_polygons_areas_pixels
    polygons_areas[image_id] = image_polygons_areas


all_polygons_area_pixels = [item for sublist in polygons_areas_pixels.values() for item in sublist]

all_polygons_area = [item for sublist in polygons_areas.values() for item in sublist]